In [ ]:
import os
import requests
import numpy as np
import librosa
import sounddevice as sd
import tensorflow as tf
import tensorflow_hub as hub
import threading
import cv2
from scipy.io.wavfile import write
from scipy.signal import butter, lfilter
from ultralytics import YOLO
from datetime import datetime
from deepface import DeepFace
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient
from socketio import Client

# Initialize Socket.IO client with logging
sio = Client(logger=True, engineio_logger=True)



# Initialize MongoDB client
client = MongoClient("mongodb://localhost:27017/")
db = client["SurveilXpertDB"]
alerts_collection = db["alerts"]

# === Model Loading ===
yolo_model_path = 'Yolo_nano_weights.pt'

# Load models
audio_model = tf.keras.models.load_model("best_model1.keras")

# Download YAMNet model locally if not already downloaded
yamnet_model_path = "yamnet_model"
if not os.path.exists(yamnet_model_path):
    os.makedirs(yamnet_model_path)
    
    # Download the model
    url = "https://tfhub.dev/google/yamnet/1?tf-hub-format=compressed"
    response = requests.get(url)
    with open(os.path.join(yamnet_model_path, "yamnet.tar.gz"), "wb") as f:
        f.write(response.content)
    
    # Extract the model
    with tarfile.open(os.path.join(yamnet_model_path, "yamnet.tar.gz"), "r:gz") as tar:
        tar.extractall(path=yamnet_model_path)

# Load YAMNet model from local path
yamnet_model = hub.load(yamnet_model_path)
# yamnet_model = hub.load('https://www.kaggle.com/models/google/yamnet/tensorFlow2/yamnet/1?tfhub-redirect=true')
yolo_model = YOLO(yolo_model_path)

# Ensure required directories exist
os.makedirs("anomalous_videos", exist_ok=True)
os.makedirs("anomalous_audio", exist_ok=True)
os.makedirs("unknown_faces", exist_ok=True)

# Index-to-label mapping for YAMNet
index_to_label = {
    0: "Emergency_alert_sound",
    1: "Explosions",
    2: "Gunshots",
    3: "Human screams",
    4: "Bottles breaking",
    5: "Dog bark",
    6: "Human Conversation",
}

# Noise reduction using a Butterworth filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut=300.0, highcut=3400.0, fs=16000, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Preprocess and generate YAMNet embeddings
def preprocess_audio(audio_waveform, target_length=16000 * 5):
    audio_waveform = bandpass_filter(audio_waveform)
    if len(audio_waveform) < target_length:
        audio_waveform = np.pad(audio_waveform, (0, target_length - len(audio_waveform)))
    else:
        audio_waveform = audio_waveform[:target_length]
    audio_waveform = audio_waveform.astype(np.float32)
    audio_waveform /= np.max(np.abs(audio_waveform))
    return audio_waveform

def predict_audio_from_stream(audio_data, sr, confidence_threshold=0.7):
    processed_audio = preprocess_audio(audio_data)
    _, yamnet_embeddings, _ = yamnet_model(processed_audio)
    avg_embedding = tf.reduce_mean(yamnet_embeddings, axis=0).numpy().reshape(1, -1)
    prediction = audio_model.predict(avg_embedding)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class_index]
    if confidence >= confidence_threshold:
        predicted_class_name = index_to_label.get(predicted_class_index, "Unknown")
        return predicted_class_name, confidence
    return "Unknown", confidence

# Audio capturing and processing thread
def process_audio_stream(duration=5, sample_rate=16000):
    print("[INFO] Starting audio detection thread...")
    while True:
        print("[INFO] Recording audio...")
        audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
        sd.wait()
        audio_data = audio_data.flatten()

        predicted_class, confidence = predict_audio_from_stream(audio_data, sample_rate)

        if predicted_class != "Unknown":
            print(f"[{datetime.now()}] Detected audio anomaly: {predicted_class} (Confidence: {confidence:.2f})")

            if predicted_class == "Human Conversation":
                print("[INFO] Detected 'Human Conversation'. Audio will not be recorded.")
                continue
            if predicted_class == "Explosions":
                continue

            audio_filename = f"anomalous_audio/{predicted_class}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
            write(audio_filename, sample_rate, (audio_data * 32767).astype(np.int16))
            print(f"[INFO] Audio saved to: {audio_filename}")

            alert = {
                "type": "audio",
                "anomaly": predicted_class,
                "timestamp": datetime.now(),
                "file_path": audio_filename
            }
            alerts_collection.insert_one(alert)
            try:
                print("[INFO] Emitting alert...")
                sio.emit('new_alert', alert)
                print("[INFO] Alert emitted successfully")
            except Exception as e:
                print(f"[ERROR] Failed to emit alert: {e}")

# Face recognition setup
known_faces_dir = "known_faces"
os.makedirs(known_faces_dir, exist_ok=True)
known_encodings = []
known_names = []

# Load known faces
def load_known_faces():
    for name in os.listdir(known_faces_dir):
        person_dir = os.path.join(known_faces_dir, name)
        if not os.path.isdir(person_dir):
            continue
        for file in os.listdir(person_dir):
            file_path = os.path.join(person_dir, file)
            try:
                encoding = DeepFace.represent(img_path=file_path, model_name='Facenet')[0]['embedding']
                known_encodings.append(encoding)
                known_names.append(name)
            except Exception as e:
                print(f"Error loading face {file_path}: {e}")

load_known_faces()

def recognize_faces(frame):
    try:
        detections = DeepFace.find(img_path=frame, db_path=known_faces_dir, model_name='Facenet', enforce_detection=False)
        if detections and not detections[0].empty:
            return detections[0].iloc[0]['identity']
        else:
            return "Unknown"
    except Exception as e:
        print(f"Error during face recognition: {e}")
        return "Error"

def is_face_saved(face_encoding, threshold=0.6):
    for known_encoding in known_encodings:
        similarity = cosine_similarity([face_encoding], [known_encoding])[0][0]
        if similarity > threshold:
            return True
    return False

# Save the unknown face to the folder
def save_unknown_face(frame):
    try:
        encoding = DeepFace.represent(img_path=frame, model_name='Facenet')[0]['embedding']
        if not is_face_saved(encoding):
            face_filename = f"unknown_faces/unknown_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            cv2.imwrite(face_filename, frame)
            alert = {
                "type": "face",
                "anomaly": "Unknown Face detected",
                "timestamp": datetime.now(),
                "file_path": face_filename
            }
            alerts_collection.insert_one(alert)
            try:
                print("[INFO] Emitting face alert...")
                sio.emit('new_alert', alert)
                print("[INFO] Face alert emitted successfully")
            except Exception as e:
                print(f"[ERROR] Failed to emit face alert: {e}")
            known_encodings.append(encoding)
        else:
            print(f"[INFO] Duplicate unknown face detected, not saving.")
    except Exception as e:
        print(f"Error saving unknown face: {e}")

# Video threat detection
def process_video_stream(video_source=0):
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Could not open video stream")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    out = None
    recording = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Warning: Failed to capture frame, skipping...")
            continue

        # Perform face recognition
        face_result = recognize_faces(frame)
        if face_result == "Unknown":
            print(f"[{datetime.now()}] Unknown face detected!")
            save_unknown_face(frame)

        # Perform object detection
        results = yolo_model.predict(frame, conf=0.75, verbose=False)

        anomalies_detected = False
        if results[0].boxes:
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].numpy())
                cls_id = int(box.cls)
                confidence = box.conf.item()
                label = yolo_model.names[cls_id]

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(
                    frame,
                    f"{label} ({confidence:.2f})",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 255),
                    2,
                )

                print(f"[{datetime.now()}] Detected: {label} (Confidence: {confidence:.2f})")
                if label in ["violence", "weaponized"]:
                    anomalies_detected = True
                    if not recording:
                        video_filename = f"anomalous_videos/{label}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
                        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))
                        recording = True
                        print(f"[{datetime.now()}] Recording started: {video_filename}")
                        alert = {
                            "type": "video",
                            "anomaly": label,
                            "confidence": confidence,
                            "timestamp": datetime.now(),
                            "file_path": video_filename
                        }
                        alerts_collection.insert_one(alert)
                        try:
                            print("[INFO] Emitting video alert...")
                            sio.emit('new_alert', alert)
                            print("[INFO] Video alert emitted successfully")
                        except Exception as e:
                            print(f"[ERROR] Failed to emit video alert: {e}")

            if recording and out:
                out.write(frame)

        if not anomalies_detected and recording:
            print(f"[{datetime.now()}] Anomaly ended. Stopping recording.")
            recording = False
            if out:
                out.release()
                out = None

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit requested. Stopping program.")
            break

    if recording and out:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

# Unified system
def unified_system(video_source=0):
    video_thread = threading.Thread(target=process_video_stream, args=(video_source,))
    audio_thread = threading.Thread(target=process_audio_stream)

    video_thread.start()
    audio_thread.start()

    video_thread.join()
    audio_thread.join()

if __name__ == "__main__":
    unified_system()